In [41]:
import torch
import lightning
from torchvision import models
from torch.utils.data import DataLoader

In [ ]:
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

In [11]:
from glob import glob
import webdataset as wds
from torchvision import transforms
import torch

transform = transforms.ToTensor()

def parse_labels(label_bytes):
    return torch.tensor([int(x) for x in label_bytes.decode("utf-8").split()])

wds.autodecode.decoders["cls"] = lambda data: data  # keep bytes

dataset = (
    wds.WebDataset(glob("data/train/*.tar"))
    .decode("pil")
    .to_tuple("__key__", "jpg", "cls")
    .map_tuple(lambda k: k,            # leave key as string
               lambda img: transform(img),  # image → tensor
               parse_labels)                # labels → tensor
)


In [13]:
from tqdm import tqdm
for sample in dataset:
    print(sample)
    break

('20027383_003_000', tensor([[[0.4588, 0.3098, 0.1529,  ..., 0.6745, 0.6824, 0.6863],
         [0.1922, 0.0941, 0.0118,  ..., 0.2314, 0.2353, 0.2392],
         [0.0275, 0.0000, 0.0000,  ..., 0.0039, 0.0039, 0.0039],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.4588, 0.3098, 0.1529,  ..., 0.6745, 0.6824, 0.6863],
         [0.1922, 0.0941, 0.0118,  ..., 0.2314, 0.2353, 0.2392],
         [0.0275, 0.0000, 0.0000,  ..., 0.0039, 0.0039, 0.0039],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[0.4588, 0.3098, 0.1529,  ..., 0.6745, 0.6824, 0.6863],
         [0.1922, 0.0941, 0.0118,  ..., 0.2314, 0.2353, 0.2392],
         [0.0275, 0.0000, 0.0000,  ..

In [ ]:
from torch.utils.data import DataLoader
loader = DataLoader(dataset=dataset, batch_size=32,)

for batch in tqdm(loader):
    pass



0it [00:05, ?it/s]


RuntimeError: DataLoader worker (pid(s) 25440, 24124, 16680, 16496) exited unexpectedly